### **Draft con primeros pasos ETL dataset [Cost of Living](https://www.kaggle.com/datasets/mvieira101/global-cost-of-living/code)**

1. Extracción datos raw del csv Cost of Living. 
2. Análisis explotario superficial de variables, volumen de nulos por variable y datos estadísticos generales. 
3. Elaboración de mapping de variables: incorporamos csv con nuevos nombres de variables (nombres descriptivos) y tipo de coste de esas variable. 
4. Incorporamos ese mapping en csv a la carpeta /src/data/ y /src/norebooks/data, además del README del respositorio. 
5. Renombramos columnas de variables para disponer de un naming de negocio que ayude al EDA posterior. 
6. Análisis y limpieza de nulos por variable para disponer de un dataset totalmente preparado para el análisis posterior. 
7. Guardado del CSV limpio tras el proceso de ETL para su posterior análisis. 
7. Proceso de ingeniería de características para tener variables que permitan un análisis mejor ponderado.

#### **1. Extracción de datos raw del dataset

Duplicamos la carpeta de los datasets dentro de la carpeta /src/notebooks/ para facilitar la extracción de los datos

In [23]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

pd.options.mode.copy_on_write = True # CoW por defecto a partir de pandas 3.0.0 

In [24]:
# Extraigo y examino algunos de los datos que devuelve el dataset
# Aquí puede haber una primer decisión que sea trabajar con variables con nombres agnóstico o cambiarlos por los nombres descriptivos
# En principio tiene sentido cambiarlos por los nombres descriptivos si queremos hacer un análisis exploratorio para resolver problemas de negocio reales

df_cost = pd.read_csv("./data/cost-of-living.csv")
df_cost

,city,country,x1,x2,x3,x4,x5,x6,x7,x8,...,x47,x48,x49,x50,x51,x52,x53,x54,x55,data_quality
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,110.36,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,123.51,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,43.89,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,41.17,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,36.50,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,Peterborough,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4952,Georgetown,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4953,Ixtapa Zihuatanejo,Mexico,5.16,30.94,12.89,0.98,NaN,1.80,0.62,0.41,...,103.14,412.55,257.84,515.69,412.55,NaN,NaN,NaN,NaN,0
4954,Iqaluit,Canada,29.65,74.27,13.71,6.67,8.89,3.71,3.52,4.08,...,NaN,NaN,NaN,2964.60,2964.60,NaN,NaN,NaN,6.53,0


#### **2. Análisis exploratorio de variables del dataset

Visualizamos algunos datos generales del dataset (especificidad, nulos y datos estadísticos generales)

In [25]:
# Visualizamos 10 filas del dataset para hacernos una idea de la relación entre variables (columnas)

df_cost.head(10)

,city,country,x1,x2,x3,x4,x5,x6,x7,x8,...,x47,x48,x49,x50,x51,x52,x53,x54,x55,data_quality
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,110.36,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,123.51,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,43.89,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,41.17,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,36.50,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1
5,Dhaka,Bangladesh,1.95,11.71,4.88,5.85,5.12,1.95,0.29,0.16,...,41.53,142.09,87.79,347.57,208.50,1119.98,571.72,280.73,9.26,1
6,Osaka,Japan,7.45,48.39,5.36,3.35,3.72,3.28,1.09,0.81,...,132.61,674.96,376.14,1737.21,993.17,8043.38,4825.58,2322.46,1.49,1
7,Jakarta,Indonesia,2.59,22.69,3.57,2.06,3.24,2.23,0.61,0.27,...,79.85,505.59,277.43,1172.14,615.04,2632.80,1241.09,509.12,9.05,1
8,Shenzhen,China,4.27,28.47,4.98,1.14,3.99,4.20,0.47,0.34,...,106.77,738.75,435.07,1682.30,886.16,17898.73,8091.57,1572.22,4.99,1
9,Kinshasa,Congo,15.11,42.63,10.08,1.74,2.50,4.35,2.78,0.84,...,93.33,2000.00,725.00,4500.00,1160.00,6170.63,933.33,400.00,19.33,0


In [26]:
df_cost.info() 

# Aquí observamos que hay variables que tienen nulos 
# Las columnas de ciudad, país y data_quality no tiene números faltantes --> total: 4956 non-null


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4956 entries, 0 to 4955
Data columns (total 58 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          4956 non-null   object 
 1   country       4956 non-null   object 
 2   x1            4528 non-null   float64
 3   x2            4505 non-null   float64
 4   x3            4622 non-null   float64
 5   x4            4460 non-null   float64
 6   x5            4516 non-null   float64
 7   x6            4612 non-null   float64
 8   x7            4511 non-null   float64
 9   x8            4640 non-null   float64
 10  x9            4578 non-null   float64
 11  x10           4543 non-null   float64
 12  x11           4613 non-null   float64
 13  x12           4449 non-null   float64
 14  x13           4478 non-null   float64
 15  x14           4398 non-null   float64
 16  x15           4390 non-null   float64
 17  x16           4584 non-null   float64
 18  x17           4575 non-null 

In [27]:
df_cost.describe() # Aquí vemos algunos datos estadísticos generales del dataset

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x47,x48,x49,x50,x51,x52,x53,x54,x55,data_quality
count,4528.000000,4505.000000,4622.000000,4460.000000,4516.000000,4612.000000,4511.000000,4640.000000,4578.000000,4543.000000,...,4403.000000,3593.000000,3525.000000,3476.000000,3444.000000,2729.000000,2653.000000,3524.000000,3950.000000,4956.000000
mean,10.346705,43.357811,6.929697,3.379774,3.942465,2.754952,1.523873,1.119804,1.148554,1.612855,...,85.976357,711.574943,560.159957,1253.982914,974.082854,3235.270854,2341.933102,1821.186305,6.518030,0.186239
std,7.157058,25.528312,2.704435,2.105957,2.100146,1.358821,0.907025,0.755257,0.517186,1.089164,...,40.434524,648.869301,527.492342,1219.395297,921.331271,6258.792897,3570.623878,1631.465580,5.413828,0.389339
min,0.450000,3.250000,1.080000,0.330000,0.310000,0.220000,0.110000,0.070000,0.260000,0.050000,...,7.600000,21.700000,12.200000,61.510000,27.120000,111.010000,49.240000,18.000000,0.780000,0.000000
25%,4.310000,22.440000,4.980000,1.600000,2.200000,1.580000,0.770000,0.480000,0.830000,0.780000,...,58.430000,256.240000,171.050000,465.200000,325.710000,1120.110000,790.310000,478.980000,3.000000,0.000000
50%,9.600000,40.000000,6.990000,3.000000,3.690000,2.630000,1.480000,1.050000,1.045000,1.320000,...,84.260000,526.870000,403.940000,921.075000,712.830000,2107.490000,1580.620000,1290.840000,5.200000,0.000000
75%,15.000000,60.000000,8.500000,5.000000,5.270000,3.770000,2.110000,1.580000,1.320000,2.200000,...,106.770000,992.610000,790.310000,1710.225000,1369.870000,3843.380000,2798.250000,2770.855000,8.330000,0.000000
max,57.140000,213.690000,22.130000,20.600000,17.500000,10.000000,8.000000,5.850000,6.810000,8.820000,...,542.740000,12608.830000,8989.370000,27397.380000,17868.180000,240963.670000,80321.220000,12821.400000,61.330000,1.000000


#### **3. Elaboración de mapping de variables del dataset**

Generamos un mapa que nos permita renombrar las variables del dataset Cost of Living con nombres que ayuden al análisis de negocio

In [28]:
# Cargamos un CSV con los nombres de las variables nuevos y además incorporamos una columna descriptiva del tipo de coste. 
# La columna de tipo de coste nos puede ayudar a clusterizar las variables, agruparlas y calcular métricas interesantes para el análisis 

pd.read_csv("./data/cost-of-living-vars-map.csv")

,original_name_var,name_var,description_var,cost_type
0,city,city_name,City name,NaN
1,country,country_name,Country name,NaN
2,x1,meal_inexpensive_restaurant,"Meal, Inexpensive Restaurant (USD)",Restaurants and Beverages
3,x2,meal_midrange_restaurant_2p,"Meal for 2 People, Mid-Range Restaurant, Three...",Restaurants and Beverages
4,x3,mcmeal_fastfood,McMeal at McDonald’s (or Equivalent Combo Meal...,Restaurants and Beverages
5,x4,beer_domestic_restaurant_0_5l,"Domestic Beer (0.5 liter draught, in restauran...",Restaurants and Beverages
6,x5,beer_imported_restaurant_0_33l,"Imported Beer (0.33 liter bottle, in restauran...",Restaurants and Beverages
7,x6,cappuccino_restaurant,"Cappuccino (regular, in restaurant) (USD)",Restaurants and Beverages
8,x7,soda_restaurant_0_33l,"Coke/Pepsi (0.33 liter bottle, in restaurant) ...",Restaurants and Beverages
9,x8,water_restaurant_0_33l,"Water (0.33 liter bottle, in restaurant) (USD)",Restaurants and Beverages


In [29]:
# Una vez pulido el mapping nuevo de variables, lo instancionamos en un dataset 

df_vars_cost = pd.read_csv("./data/cost-of-living-vars-map.csv")

#### **5. Renombramos columnas de variables para disponer de un naming de negocio**

Renombramos todas las columnas usando la nomenclatura estándar `snake case`

In [30]:
# Preparamos el diccionario de mapeo: de 'original_name_var' a 'name_var'
# Usamos 'zip' para emparejar la columna antigua con la nueva

rename_vars_dict = dict(zip(df_vars_cost['original_name_var'], df_vars_cost['name_var']))

In [31]:
# Renombramos columnas con método rename
# Visualizamos 20 filas del dataset para ver si está todo correcto comparado con el original --> parece que está todo OK 

df_cost_rename = df_cost.rename(columns=rename_vars_dict)
df_cost_rename.head(20)

,city_name,country_name,meal_inexpensive_restaurant,meal_midrange_restaurant_2p,mcmeal_fastfood,beer_domestic_restaurant_0_5l,beer_imported_restaurant_0_33l,cappuccino_restaurant,soda_restaurant_0_33l,water_restaurant_0_33l,...,leather_business_shoes,rent_1br_city_center,rent_1br_outside_center,rent_3br_city_center,rent_3br_outside_center,price_sqm_city_center,price_sqm_outside_center,avg_net_salary,mortgage_interest_rate_20y,data_quality_flag
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,110.36,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,123.51,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,43.89,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,41.17,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,36.50,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1
5,Dhaka,Bangladesh,1.95,11.71,4.88,5.85,5.12,1.95,0.29,0.16,...,41.53,142.09,87.79,347.57,208.50,1119.98,571.72,280.73,9.26,1
6,Osaka,Japan,7.45,48.39,5.36,3.35,3.72,3.28,1.09,0.81,...,132.61,674.96,376.14,1737.21,993.17,8043.38,4825.58,2322.46,1.49,1
7,Jakarta,Indonesia,2.59,22.69,3.57,2.06,3.24,2.23,0.61,0.27,...,79.85,505.59,277.43,1172.14,615.04,2632.80,1241.09,509.12,9.05,1
8,Shenzhen,China,4.27,28.47,4.98,1.14,3.99,4.20,0.47,0.34,...,106.77,738.75,435.07,1682.30,886.16,17898.73,8091.57,1572.22,4.99,1
9,Kinshasa,Congo,15.11,42.63,10.08,1.74,2.50,4.35,2.78,0.84,...,93.33,2000.00,725.00,4500.00,1160.00,6170.63,933.33,400.00,19.33,0


In [32]:
# Hacemos una copia del dataset y lo renombramos antes de pasar a limpiar valores nulos
# En este punto debemos decidir también si queremos analizar variables en USD o buscamos introducir variables en EUR

df_cost_living = df_cost_rename.copy()
df_cost_living

,city_name,country_name,meal_inexpensive_restaurant,meal_midrange_restaurant_2p,mcmeal_fastfood,beer_domestic_restaurant_0_5l,beer_imported_restaurant_0_33l,cappuccino_restaurant,soda_restaurant_0_33l,water_restaurant_0_33l,...,leather_business_shoes,rent_1br_city_center,rent_1br_outside_center,rent_3br_city_center,rent_3br_outside_center,price_sqm_city_center,price_sqm_outside_center,avg_net_salary,mortgage_interest_rate_20y,data_quality_flag
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,110.36,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,123.51,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,43.89,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,41.17,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,36.50,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,Peterborough,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4952,Georgetown,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4953,Ixtapa Zihuatanejo,Mexico,5.16,30.94,12.89,0.98,NaN,1.80,0.62,0.41,...,103.14,412.55,257.84,515.69,412.55,NaN,NaN,NaN,NaN,0
4954,Iqaluit,Canada,29.65,74.27,13.71,6.67,8.89,3.71,3.52,4.08,...,NaN,NaN,NaN,2964.60,2964.60,NaN,NaN,NaN,6.53,0


#### **6. Análisis y limpieza de nulos para disponer de un dataset limpio**

Analizamos los nulos por cada variable. 

Valoramos opciones correctas para su limpieza antes de pasar a la creación de variables calculadas: 

6.1. Copia del dataset y revisión de qué filas tienen nulos por encima del 70%.
6.2. Eliminación de variables con alto % de nulos y baja relevancia. 
6.3. Estrategia de medianas por niveles geográficos para mantener el valor analítico del dataset: 
- 1. Establecemos primero medianas por país. 
- 2. Después añadimos una columna continente que ahora no existe, pero nos ayuda a evitar aplicar medianas globales. 
- 3. Por última establecemos medianas globales si no queda otro remedio. 

Las medianas por país tiene más sentido porque completar un nulo con la mediana del resto de ciudades de ese país puede ser más preciso. Por continente aún menos precios. Globales, aún menos. 


In [33]:
df_cost_living.info() # Buscamos datos de valores nulos en el dataset con los nuevos nombres de variables asignadas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4956 entries, 0 to 4955
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   city_name                            4956 non-null   object 
 1   country_name                         4956 non-null   object 
 2   meal_inexpensive_restaurant          4528 non-null   float64
 3   meal_midrange_restaurant_2p          4505 non-null   float64
 4   mcmeal_fastfood                      4622 non-null   float64
 5   beer_domestic_restaurant_0_5l        4460 non-null   float64
 6   beer_imported_restaurant_0_33l       4516 non-null   float64
 7   cappuccino_restaurant                4612 non-null   float64
 8   soda_restaurant_0_33l                4511 non-null   float64
 9   water_restaurant_0_33l               4640 non-null   float64
 10  milk_1l                              4578 non-null   float64
 11  bread_white_500g              

#### **6.1 Copia del dataset y eliminación de filas con datos nulos excesivos**

Antes de iniciar la limpieza, guardamos una copia del dataset.

El criterio de eliminación por filas es que tengan un volumen total de valores NaN igual o por encima del 70%. 

In [34]:
# Guardamos copia del dataset original antes de esta fase de la limpieza.

df_cost_original = df_cost_living.copy()

# Calculamos el porcentaje de nulos por fila, lógicamente solo para las columnas numéricas.
# Quedan excluidas del cálculo las variables city_name, country_name que nunca tienen nulos.

num_cols = df_cost_living.select_dtypes(include=['float64', 'int64']).columns
nulls_row = df_cost_living[num_cols].isnull().sum(axis=1) / len(num_cols) * 100

# Visualizamos la distribución de nulos por fila antes del filtrado
# Miramos volumen de filas con 70%, 50% y 30% de nulos. 
# A partir de ahí podemos decidir si ser más restrictivo merma en exceso el dataset. 

print("Distribución de nulos por fila (antes del filtrado)")
print(f"Total filas: {len(df_cost_living)}")
print(f"Filas con >70% nulos: {(nulls_row > 70).sum()}")
print(f"Filas con >50% nulos: {(nulls_row > 50).sum()}")
print(f"Filas con >30% nulos: {(nulls_row > 30).sum()}")

Distribución de nulos por fila (antes del filtrado)
Total filas: 4956
Filas con >70% nulos: 214
Filas con >50% nulos: 469
Filas con >30% nulos: 933


In [35]:
# Eliminamos solo las filas con >70% de valores nulos para no eliminar filas en exceso. 
# Estas son ciudades con un volumen de datos insuficientes para el análisis

df_cost_living = df_cost_living[nulls_row <= 70]

In [36]:
df_cost_living.info() # Comprobamos filas eliminadas. Total de filas en este momento: 4742 filas

<class 'pandas.core.frame.DataFrame'>
Index: 4742 entries, 0 to 4955
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   city_name                            4742 non-null   object 
 1   country_name                         4742 non-null   object 
 2   meal_inexpensive_restaurant          4468 non-null   float64
 3   meal_midrange_restaurant_2p          4443 non-null   float64
 4   mcmeal_fastfood                      4551 non-null   float64
 5   beer_domestic_restaurant_0_5l        4404 non-null   float64
 6   beer_imported_restaurant_0_33l       4455 non-null   float64
 7   cappuccino_restaurant                4548 non-null   float64
 8   soda_restaurant_0_33l                4457 non-null   float64
 9   water_restaurant_0_33l               4565 non-null   float64
 10  milk_1l                              4532 non-null   float64
 11  bread_white_500g                   

In [37]:
# Aquí miramos volumen total de nulos por columnas (variables), una vez eliminas las filas >=70% de nulos (ciudades)
# Quizás vemos alguna fila que no es relevante para el análisis de nomadismo digital con un volumen de nulos muy alto. 
# Lo lógico es mantener todas las que tienen que ver con coste real de vida diaria y eliminar las que no.

df_cost_living.isna().sum()

city_name                                 0
country_name                              0
meal_inexpensive_restaurant             274
meal_midrange_restaurant_2p             299
mcmeal_fastfood                         191
beer_domestic_restaurant_0_5l           338
beer_imported_restaurant_0_33l          287
cappuccino_restaurant                   194
soda_restaurant_0_33l                   285
water_restaurant_0_33l                  177
milk_1l                                 210
bread_white_500g                        244
rice_white_1kg                          166
eggs_12                                 334
cheese_local_1kg                        289
chicken_fillet_1kg                      366
beef_1kg                                369
apples_1kg                              184
bananas_1kg                             198
oranges_1kg                             248
tomatoes_1kg                            274
potatoes_1kg                            242
onions_1kg                      

#### **6.2 Eliminación de variable con alto % de nulos y baja relevancia**

Del análisis previo, sólo una variable tiene un % de nulos muy alto y no sirve en exceso para el análisis: `tennis_court_1h_weekend`.

Esta variable tiene un 48% de nulos. 

Hay otra variables como `price_sqm_city_center `, `price_sqm_outside_center`, `price_sqm_` o `public_transport_monthly_pass` que tienen un volumen de nulos también my alto pero sí son relevantes para el EDA. Las mantenemos.

In [38]:
df_cost_living = df_cost_living.drop(columns=['tennis_court_1h_weekend'])

print(f"Columnas totales: {len(df_cost_living.columns)}")

Columnas totales: 57


#### **6.3 Corrección de nulos con medianas: añadimos una columna continente**

Añadimos una columna `continent` que agrupa los países por continente. 

Esta columna servirá como opción usada cuando la normalización de nulos no pueda hacerse con país. 

Los continentes será Europe, Asia, North America, South America, Africa, Oceania.

In [43]:
# Primero identificamos todos los países únicos en el dataset

unique_countries = df_cost_living['country_name'].unique()
print(f"Países únicos en el dataset: {len(unique_countries)}")

# Imprimo todos los países del dataset para luego asignarles continente

print(sorted(unique_countries))

Países únicos en el dataset: 214
['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia And Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Guadeloup

In [44]:
# Le pido a CHATGPT que me haga un diccionario asociando los 6 continente a cada uno de los países de la lista

country_to_continent = {
    # Europe
    'Albania': 'Europe', 'Andorra': 'Europe', 'Austria': 'Europe', 'Belarus': 'Europe',
    'Belgium': 'Europe', 'Bosnia And Herzegovina': 'Europe', 'Bulgaria': 'Europe',
    'Croatia': 'Europe', 'Cyprus': 'Europe', 'Czech Republic': 'Europe', 'Denmark': 'Europe',
    'Estonia': 'Europe', 'Finland': 'Europe', 'France': 'Europe', 'Germany': 'Europe',
    'Greece': 'Europe', 'Hungary': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe',
    'Italy': 'Europe', 'Kosovo': 'Europe', 'Latvia': 'Europe', 'Liechtenstein': 'Europe',
    'Lithuania': 'Europe', 'Luxembourg': 'Europe', 'Malta': 'Europe', 'Moldova': 'Europe',
    'Monaco': 'Europe', 'Montenegro': 'Europe', 'Netherlands': 'Europe', 'North Macedonia': 'Europe',
    'Norway': 'Europe', 'Poland': 'Europe', 'Portugal': 'Europe', 'Romania': 'Europe',
    'Russia': 'Europe', 'San Marino': 'Europe', 'Serbia': 'Europe', 'Slovakia': 'Europe',
    'Slovenia': 'Europe', 'Spain': 'Europe', 'Sweden': 'Europe', 'Switzerland': 'Europe',
    'Ukraine': 'Europe', 'United Kingdom': 'Europe',
    # Europe - Territorios y dependencias
    'Gibraltar': 'Europe', 'Isle Of Man': 'Europe', 'Jersey': 'Europe',
    'Faroe Islands': 'Europe', 'Kosovo (Disputed Territory)': 'Europe', 'Vatican City': 'Europe',
    
    # Asia
    'Afghanistan': 'Asia', 'Armenia': 'Asia', 'Azerbaijan': 'Asia', 'Bahrain': 'Asia',
    'Bangladesh': 'Asia', 'Bhutan': 'Asia', 'Brunei': 'Asia', 'Cambodia': 'Asia',
    'China': 'Asia', 'Georgia': 'Asia', 'Hong Kong': 'Asia', 'India': 'Asia',
    'Indonesia': 'Asia', 'Iran': 'Asia', 'Iraq': 'Asia', 'Israel': 'Asia',
    'Japan': 'Asia', 'Jordan': 'Asia', 'Kazakhstan': 'Asia', 'Kuwait': 'Asia',
    'Kyrgyzstan': 'Asia', 'Laos': 'Asia', 'Lebanon': 'Asia', 'Macau': 'Asia',
    'Malaysia': 'Asia', 'Maldives': 'Asia', 'Mongolia': 'Asia', 'Myanmar': 'Asia',
    'Nepal': 'Asia', 'North Korea': 'Asia', 'Oman': 'Asia', 'Pakistan': 'Asia',
    'Palestine': 'Asia', 'Philippines': 'Asia', 'Qatar': 'Asia', 'Saudi Arabia': 'Asia',
    'Singapore': 'Asia', 'South Korea': 'Asia', 'Sri Lanka': 'Asia', 'Syria': 'Asia',
    'Taiwan': 'Asia', 'Tajikistan': 'Asia', 'Thailand': 'Asia', 'Timor-Leste': 'Asia',
    'Turkey': 'Asia', 'Turkmenistan': 'Asia', 'United Arab Emirates': 'Asia',
    'Uzbekistan': 'Asia', 'Vietnam': 'Asia', 'Yemen': 'Asia',
    
    # North America (incluye Caribe y Centroamérica)
    'Bahamas': 'North America', 'Barbados': 'North America', 'Belize': 'North America',
    'Canada': 'North America', 'Costa Rica': 'North America', 'Cuba': 'North America',
    'Dominican Republic': 'North America', 'El Salvador': 'North America',
    'Guatemala': 'North America', 'Haiti': 'North America', 'Honduras': 'North America',
    'Jamaica': 'North America', 'Mexico': 'North America', 'Nicaragua': 'North America',
    'Panama': 'North America', 'Puerto Rico': 'North America', 'Trinidad And Tobago': 'North America',
    'United States': 'North America',
    # North America - Caribe y territorios
    'Martinique': 'North America', 'Curacao': 'North America', 'Saint Lucia': 'North America',
    'Bermuda': 'North America', 'Aruba': 'North America', 'Saint Kitts And Nevis': 'North America',
    'Saint Vincent And The Grenadines': 'North America', 'Antigua And Barbuda': 'North America',
    'Dominica': 'North America', 'Montserrat': 'North America', 'Sint Maarten': 'North America',
    'British Virgin Islands': 'North America', 'Anguilla': 'North America',
    'Guadeloupe': 'North America', 'Greenland': 'North America',
    
    # South America
    'Argentina': 'South America', 'Bolivia': 'South America', 'Brazil': 'South America',
    'Chile': 'South America', 'Colombia': 'South America', 'Ecuador': 'South America',
    'Guyana': 'South America', 'Paraguay': 'South America', 'Peru': 'South America',
    'Suriname': 'South America', 'Uruguay': 'South America', 'Venezuela': 'South America',
    # South America - Territorios
    'French Guiana': 'South America', 'Falkland Islands': 'South America',
    
    # Africa
    'Algeria': 'Africa', 'Angola': 'Africa', 'Benin': 'Africa', 'Botswana': 'Africa',
    'Burkina Faso': 'Africa', 'Burundi': 'Africa', 'Cameroon': 'Africa', 'Cape Verde': 'Africa',
    'Central African Republic': 'Africa', 'Chad': 'Africa', 'Comoros': 'Africa',
    'Congo': 'Africa', 'Djibouti': 'Africa', 'Egypt': 'Africa', 'Equatorial Guinea': 'Africa',
    'Eritrea': 'Africa', 'Eswatini': 'Africa', 'Ethiopia': 'Africa', 'Gabon': 'Africa',
    'Gambia': 'Africa', 'Ghana': 'Africa', 'Guinea': 'Africa', 'Ivory Coast': 'Africa',
    'Kenya': 'Africa', 'Lesotho': 'Africa', 'Liberia': 'Africa', 'Libya': 'Africa',
    'Madagascar': 'Africa', 'Malawi': 'Africa', 'Mali': 'Africa', 'Mauritania': 'Africa',
    'Mauritius': 'Africa', 'Morocco': 'Africa', 'Mozambique': 'Africa', 'Namibia': 'Africa',
    'Niger': 'Africa', 'Nigeria': 'Africa', 'Rwanda': 'Africa', 'Senegal': 'Africa',
    'Seychelles': 'Africa', 'Sierra Leone': 'Africa', 'Somalia': 'Africa', 'South Africa': 'Africa',
    'South Sudan': 'Africa', 'Sudan': 'Africa', 'Tanzania': 'Africa', 'Togo': 'Africa',
    'Tunisia': 'Africa', 'Uganda': 'Africa', 'Zambia': 'Africa', 'Zimbabwe': 'Africa',
    # Africa - Países y territorios adicionales
    'Guinea-Bissau': 'Africa', 'Swaziland': 'Africa', 'Sao Tome And Principe': 'Africa',
    'Reunion': 'Africa', 'Saint Helena': 'Africa',
    
    # Oceania
    'Australia': 'Oceania', 'Fiji': 'Oceania', 'New Zealand': 'Oceania',
    'Papua New Guinea': 'Oceania', 'Samoa': 'Oceania', 'Solomon Islands': 'Oceania',
    'Tonga': 'Oceania', 'Vanuatu': 'Oceania',
    # Oceania - Territorios adicionales
    'French Polynesia': 'Oceania', 'New Caledonia': 'Oceania', 'Marshall Islands': 'Oceania',
    'American Samoa': 'Oceania', 'Tuvalu': 'Oceania', 'Cook Islands': 'Oceania', 'Nauru': 'Oceania',
}

# Creamos la columna 'continent' a partir del diccionario que relaciona listado de continentes con países (country_name)

df_cost_living['continent'] = df_cost_living['country_name'].map(country_to_continent)

# Verificamos si hay países sin mapear (NaN en continent).

country_continent = df_cost_living[df_cost_living['continent'].isna()]['country_name'].unique()
if len(country_continent) > 0:
    print(f"Países sin mapear ({len(country_continent)}): {list(country_continent)}")
else:
    print("Todos los países mapeados correctamente")

# Resumen por continente.

print(f"\nDistribución por continente")
print(df_cost_living['continent'].value_counts())

Todos los países mapeados correctamente

Distribución por continente
continent
Europe           1730
North America    1338
Asia              961
Africa            311
South America     305
Oceania            97
Name: count, dtype: int64


#### **6.4 Corrección de nulos en base a los tres niveles: países, continente y global**

In [ ]:
# Columnas numéricas a imputar
cols_median = df_cost_living.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Contamos nulos antes de imputar
nulls_before = df_cost_living[cols_median].isnull().sum().sum()
print(f"Nulos antes de imputar: {nulls_before:,}")

# Nivel 1: Mediana por país
country_medians = df_cost_living.groupby('country_name')[cols_median].transform('median')
df_cost_living[cols_median] = df_cost_living[cols_median].fillna(country_medians)

nulls_after_country = df_cost_living[cols_median].isnull().sum().sum()
print(f"Imputados por país: {nulls_before - nulls_after_country:,}")

# Nivel 2: Mediana por continente
continent_medians = df_cost_living.groupby('continent')[cols_median].transform('median')
df_cost_living[cols_median] = df_cost_living[cols_median].fillna(continent_medians)

nulls_after_continent = df_cost_living[cols_median].isnull().sum().sum()
print(f"Imputados por continente: {nulls_after_country - nulls_after_continent:,}")

# Nivel 3: Mediana global
global_medians = df_cost_living[cols_median].median()
df_cost_living[cols_median] = df_cost_living[cols_median].fillna(global_medians)

nulls_after_global = df_cost_living[cols_median].isnull().sum().sum()
print(f"Imputados por global: {nulls_after_continent - nulls_after_global:,}")

# Resumen final
print(f"\n✓ Imputación completada. Nulos restantes: {nulls_after_global}")

Nulos antes de imputar: 33,493
Imputados por país: 32,913
Imputados por continente: 580
Imputados por global: 0

✓ Imputación completada. Nulos restantes: 0


In [47]:
# Listado de variables son nulos y un total de 4752 filas cada una. 
# Del volumen de nulos totales normalizados con medianas, ninguno fue a partir de datos globales. 
# El 98% de los nulos totales de todas las variables se normalizaron con las medianas por países, ni siquiera continente. 

df_cost_living.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 4742 entries, 0 to 4955
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   city_name                            4742 non-null   object 
 1   country_name                         4742 non-null   object 
 2   meal_inexpensive_restaurant          4742 non-null   float64
 3   meal_midrange_restaurant_2p          4742 non-null   float64
 4   mcmeal_fastfood                      4742 non-null   float64
 5   beer_domestic_restaurant_0_5l        4742 non-null   float64
 6   beer_imported_restaurant_0_33l       4742 non-null   float64
 7   cappuccino_restaurant                4742 non-null   float64
 8   soda_restaurant_0_33l                4742 non-null   float64
 9   water_restaurant_0_33l               4742 non-null   float64
 10  milk_1l                              4742 non-null   float64
 11  bread_white_500g                   

#### **6.5 Guardamos del dataset limpio en la carpeta de data**

In [48]:
# Exportamos el dataset limpio a CSV en ambas ubicaciones del proyecto
# Usamos index=False para no incluir el índice de pandas como columna

df_cost_living.to_csv('./data/cost-of-living-clean.csv', index=False)